In [1]:
import numpy as np
from netCDF4 import Dataset

INPUT_FILE = r"E:/GOES-R Lightning Data/2023/OR_GLM-L3-GLMF-M6_G16_s202318123590000_e202318200000000_c20231820000550.nc"  # replace

with Dataset(INPUT_FILE, "r") as ds:
    data = ds.variables["Total_Optical_energy"][:].astype(float)
    fill_value = getattr(ds.variables["Total_Optical_energy"], "_FillValue", np.nan)
    data[data == fill_value] = np.nan

    # Count valid pixels
    valid = data[~np.isnan(data) & (data > 0)]
    n_valid = valid.size
    print(f"Number of valid pixels: {n_valid}")

    # Example: top 10%
    n_top10 = int(0.10 * n_valid)
    print(f"Top 10% count: {n_top10}")

    # If you wanted to actually get them:
    sorted_vals = np.sort(valid)[::-1]  # descending
    top10_vals = sorted_vals[:n_top10]
    print(f"Top 10% min/max: {top10_vals.min()}, {top10_vals.max()}")


Number of valid pixels: 86862
Top 10% count: 8686
Top 10% min/max: 1.220776e-05, 0.00020905789


In [3]:
import random
import numpy as np
from netCDF4 import Dataset
import os

INPUT_DIR = r"E:/GOES-R Lightning Data/2023"
N_SAMPLE = 1000   # adjust (500-2000 good)
variable = "Total_Optical_energy"

files = [os.path.join(INPUT_DIR,f) for f in os.listdir(INPUT_DIR) if f.endswith(".nc")]
random.seed(42)
sample_files = random.sample(files, min(N_SAMPLE, len(files)))

vals = []
for fn in sample_files:
    try:
        with Dataset(fn,'r') as ds:
            if variable not in ds.variables:
                continue
            arr = ds.variables[variable][:].astype(float)
            fv = getattr(ds.variables[variable], "_FillValue", None)
            if fv is not None:
                arr[arr == fv] = np.nan
            # flatten only positive, non-nan values
            a = arr[~np.isnan(arr) & (arr > 0)]
            if a.size:
                vals.append(a.ravel())
    except Exception as e:
        continue

if not vals:
    raise SystemExit("No positive values found in sample")
vals = np.concatenate(vals)
for p in [90,95,97.5,99]:
    print(f"{p}th percentile: {np.percentile(vals, p)}")


C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:4859: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


90th percentile: 1.220776e-05
95th percentile: 2.441552e-05
97.5th percentile: 3.9675220000000004e-05
99th percentile: 6.561671e-05
